# Russian Synodal Bible (1885)

## Import

### Libraries

In [1]:
import os
import re
import json
import time
import numpy as np
import pandas as pd 
#import natasha
#from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, Doc

### Definitions

In [2]:
texts = './texts/fiction/'
bibleTXT = './texts/bible/bibliya-sinodalnyy-perevod.txt'
libCols = ['author','pub_year','title','text']
tokenOHCO = ['title','part_num','para_num', 'sent_num', 'token_num']
bibleOHCO = ['test', 'book', 'chap', 'verse']
bibleJSON = './texts/bible/bible.json'

## Pre-Processing

### Import Text into DF

In [3]:
with open(bibleTXT, 'r', encoding='windows-1251') as f: 
    bibleText = f.readlines()

bibliiaDf = pd.DataFrame(bibleText).rename(columns={0:'text'}).dropna()
bibliiaDf

,text
0,Библия. Синодальный перевод\n
1,Священное Писание \n
2,\n
3,\n
4,Перевод Библии на русский язык осуществлялся С...
...,...
191107,\n
191108,\n
191109,Подир — длинная одежда Иудейских первосвященни...
191110,\n


### Find Parts

In [4]:
cont_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Оглавление')].index[0]
OT_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Книги Ветхого Завета')].index[0]
NT_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Книги Нового Завета')].index[0]
end_line = bibliiaDf.loc[bibliiaDf.text.str.contains('notes')].index[0]
vz_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Ветхий Завет')].index[0]
nz_line = bibliiaDf.loc[bibliiaDf.text.str.contains('Новый Завет')].index[0]

### Tidy Up

In [5]:
bibliiaDf = bibliiaDf.loc[bibliiaDf.text != '\n']
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\xa0', '')
bibliiaDf

<ipython-input-5-13627834583f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
/usr/lib/python3.9/site-packages/pandas/core/indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
<ipython-input-5-13627834583f>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\xa0', '')


,text
0,Библия. Синодальный перевод
1,Священное Писание
4,Перевод Библии на русский язык осуществлялся С...
10,БИБЛИЯ
12,Книги Священного Писания Ветхого и Нового Завета
...,...
191093,Обращенные из язычников.
191098,54
191101,Евреи из стран языческих.
191106,55


### Make Dict of Outline

{Testament: {Book: abbr}}

In [6]:
abbrevsDf = bibliiaDf.loc[vz_line+1:OT_line-1]
abbrevsDf[['abbr', 'title']] = abbrevsDf.loc[:,'text'].str.extract(r'^(\[.*\]) (.*)$', expand=True)
abbrevsDf = abbrevsDf.drop(['text'], axis=1)
bibliiaDict = {
    'Ветхий Завет': dict(zip(abbrevsDf.loc[vz_line+1:nz_line-1].title.str.strip(' '), abbrevsDf.loc[vz_line+1:nz_line-1].abbr.str.strip(r'[|]'))), 
    'Новый Завет': dict(zip(abbrevsDf.loc[nz_line+1:OT_line-1].title.str.strip(' '), abbrevsDf.loc[nz_line+1:OT_line-1].abbr.str.strip(r'[|]')))
}
del abbrevsDf

/usr/lib/python3.9/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### Assign Testament Label

In [7]:
bibliiaDf = bibliiaDf.loc[OT_line+1:end_line-1]
bibliiaDf['test'] = np.where(bibliiaDf.index<NT_line, 'OT', 'NT')
bibliiaDf

<ipython-input-7-204d3fa67429>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bibliiaDf['test'] = np.where(bibliiaDf.index<NT_line, 'OT', 'NT')


,text,test
422,[Быт.] Первая книга Моисея. Бытие,OT
426,Глава 1,OT
431,В начале сотворил Бог небо и землю.,OT
436,"Земля же была безвидна и пуста, и тьма над без...",OT
441,И сказал Бог: да будет свет. И стал свет.,OT
...,...,...
190641,И Дух и невеста говорят: прииди! И слышавший д...,NT
190646,И я также свидетельствую всякому слышащему сло...,NT
190651,и если кто отнимет что от слов книги пророчест...,NT
190656,"Свидетельствующий сие говорит: ей, гряду скоро...",NT


### Index All Parts

In [8]:
all_chaps = bibliiaDf.loc[bibliiaDf.text.str.contains(r'Глава \d')]
bibleBooks = [x for y in bibliiaDict.values() for x in y]
bookAbbrs = [val[abbr] for key, val in bibliiaDict.items() for abbr in val]
searchAbbrs = [r'^\['+val[abbr]+'\]' for key, val in bibliiaDict.items() for abbr in val]
abbrsList = "|".join(searchAbbrs)
books_index = bibliiaDf[bibliiaDf['text'].str.contains(abbrsList)].index
books_map = list(zip(books_index, bookAbbrs))
chaps_map = list(all_chaps.itertuples(name=None))

### Assign Book Label

In [9]:
for ind in books_map: 
    bibliiaDf.loc[ind[0], 'book'] = ind[1]

bibliiaDf['book'] = bibliiaDf.book.ffill()
bibliiaDf = bibliiaDf.drop([x[0] for x in books_map])
bibliiaDf

/usr/lib/python3.9/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-9-fd74bf7fa7d3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,text,test,book
426,Глава 1,OT,Быт.
431,В начале сотворил Бог небо и землю.,OT,Быт.
436,"Земля же была безвидна и пуста, и тьма над без...",OT,Быт.
441,И сказал Бог: да будет свет. И стал свет.,OT,Быт.
446,"И увидел Бог свет, что он хорош, и отделил Бог...",OT,Быт.
...,...,...,...
190641,И Дух и невеста говорят: прииди! И слышавший д...,NT,Откр.
190646,И я также свидетельствую всякому слышащему сло...,NT,Откр.
190651,и если кто отнимет что от слов книги пророчест...,NT,Откр.
190656,"Свидетельствующий сие говорит: ей, гряду скоро...",NT,Откр.


### Assign Chapter & Verse Labels

In [10]:
for ind in chaps_map: 
    bibliiaDf.loc[ind[0], 'chap'] = ind[1]
    
bibliiaDf['chap'] = bibliiaDf.chap.ffill()
bibliiaDf = bibliiaDf.drop(x[0] for x in chaps_map)

bibliiaDf['chap'] = bibliiaDf.chap.str.extract(r'(\d{1,2})$')

bibliiaDf['verse'] = bibliiaDf.groupby(bibleOHCO[:3]).cumcount()+1

### Set Index & Remove Fluff

In [11]:
bibliiaDf = bibliiaDf.set_index(bibleOHCO)

bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].replace(r"\s\[.+\]", "", regex=True)

bibliiaDf

text
test book  chap verse                                                   
OT   Быт.  1    1                    В начале сотворил Бог небо и землю.
                2      Земля же была безвидна и пуста, и тьма над без...
                3              И сказал Бог: да будет свет. И стал свет.
                4      И увидел Бог свет, что он хорош, и отделил Бог...
                5      И назвал Бог свет днем, а тьму ночью. И был ве...
...                                                                  ...
NT   Откр. 22   17     И Дух и невеста говорят: прииди! И слышавший д...
                18     И я также свидетельствую всякому слышащему сло...
                19     и если кто отнимет что от слов книги пророчест...
                20     Свидетельствующий сие говорит: ей, гряду скоро...
                21     Благодать Господа нашего Иисуса Христа со всем...

[37343 rows x 1 columns]

### Export to JSON

In [12]:
bibliiaDf.reset_index().to_json(bibleJSON)

In [14]:
bibliiaDf.to_xml('./texts/bible/bible.xml')

AttributeError: 'DataFrame' object has no attribute 'to_xml'

## Processing

In [4]:
BibTextDf = pd.read_json(bibleJSON)[['text']]
BibLibDf = pd.read_json(bibleJSON)[bibleOHCO]
#synodalDf.index.name = 'index'
#synodalDf#.set_index(bibleOHCO)

In [5]:
BibLibDf

,test,book,chap,verse
0,OT,Быт.,1,1
1,OT,Быт.,1,2
2,OT,Быт.,1,3
3,OT,Быт.,1,4
4,OT,Быт.,1,5
...,...,...,...,...
37338,NT,Откр.,22,17
37339,NT,Откр.,22,18
37340,NT,Откр.,22,19
37341,NT,Откр.,22,20


In [6]:
doc = Doc(BibTextDf.loc[4].text)

In [7]:
doc

Doc(text='И назвал Бог свет днем, а тьму ночью. И был вечер...)

In [8]:
doc.segment(segmenter)

In [9]:
doc

Doc(text='И назвал Бог свет днем, а тьму ночью. И был вечер..., tokens=[...], sents=[...])

In [10]:
doc.tokens

[DocToken(stop=1, text='И'),
 DocToken(start=2, stop=8, text='назвал'),
 DocToken(start=9, stop=12, text='Бог'),
 DocToken(start=13, stop=17, text='свет'),
 DocToken(start=18, stop=22, text='днем'),
 DocToken(start=22, stop=23, text=','),
 DocToken(start=24, stop=25, text='а'),
 DocToken(start=26, stop=30, text='тьму'),
 DocToken(start=31, stop=36, text='ночью'),
 DocToken(start=36, stop=37, text='.'),
 DocToken(start=38, stop=39, text='И'),
 DocToken(start=40, stop=43, text='был'),
 DocToken(start=44, stop=49, text='вечер'),
 DocToken(start=49, stop=50, text=','),
 DocToken(start=51, stop=52, text='и'),
 DocToken(start=53, stop=57, text='было'),
 DocToken(start=58, stop=62, text='утро'),
 DocToken(start=62, stop=63, text=':'),
 DocToken(start=64, stop=68, text='день'),
 DocToken(start=69, stop=73, text='один'),
 DocToken(start=73, stop=74, text='.')]

In [11]:
doc.sents

[DocSent(stop=37, text='И назвал Бог свет днем, а тьму ночью.', tokens=[...]),
 DocSent(start=38, stop=74, text='И был вечер, и было утро: день один.', tokens=[...])]

In [13]:
doc.sents[0].tokens

[DocToken(stop=1, text='И'),
 DocToken(start=2, stop=8, text='назвал'),
 DocToken(start=9, stop=12, text='Бог'),
 DocToken(start=13, stop=17, text='свет'),
 DocToken(start=18, stop=22, text='днем'),
 DocToken(start=22, stop=23, text=','),
 DocToken(start=24, stop=25, text='а'),
 DocToken(start=26, stop=30, text='тьму'),
 DocToken(start=31, stop=36, text='ночью'),
 DocToken(start=36, stop=37, text='.')]

In [14]:
doc.tag_morph(morph_tagger)

In [15]:
doc.tokens

[DocToken(stop=1, text='И', pos='CCONJ'),
 DocToken(start=2, stop=8, text='назвал', pos='VERB', feats=<Perf,Masc,Ind,Sing,Past,Fin,Act>),
 DocToken(start=9, stop=12, text='Бог', pos='PROPN', feats=<Inan,Acc,Masc,Sing>),
 DocToken(start=13, stop=17, text='свет', pos='NOUN', feats=<Inan,Acc,Masc,Sing>),
 DocToken(start=18, stop=22, text='днем', pos='NOUN', feats=<Inan,Ins,Masc,Sing>),
 DocToken(start=22, stop=23, text=',', pos='PUNCT'),
 DocToken(start=24, stop=25, text='а', pos='CCONJ'),
 DocToken(start=26, stop=30, text='тьму', pos='NOUN', feats=<Inan,Acc,Fem,Sing>),
 DocToken(start=31, stop=36, text='ночью', pos='NOUN', feats=<Inan,Ins,Fem,Sing>),
 DocToken(start=36, stop=37, text='.', pos='PUNCT'),
 DocToken(start=38, stop=39, text='И', pos='CCONJ'),
 DocToken(start=40, stop=43, text='был', pos='AUX', feats=<Imp,Masc,Ind,Sing,Past,Fin,Act>),
 DocToken(start=44, stop=49, text='вечер', pos='NOUN', feats=<Inan,Nom,Masc,Sing>),
 DocToken(start=49, stop=50, text=',', pos='PUNCT'),
 DocTok

In [26]:
doc.tokens[1].feats['Aspect']

'Perf'

In [16]:
doc.sents[0].morph.print()

                   И CCONJ
              назвал VERB|Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
                 Бог PROPN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
                свет NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
                днем NOUN|Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing
                   , PUNCT
                   а CCONJ
                тьму NOUN|Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing
               ночью NOUN|Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing
                   . PUNCT


In [28]:
[token.lemmatize(morph_vocab) for token in doc.tokens]

[DocToken(stop=1, text='И', pos='CCONJ', lemma='и'),
 DocToken(start=2, stop=8, text='назвал', pos='VERB', feats=<Perf,Masc,Ind,Sing,Past,Fin,Act>, lemma='назвать'),
 DocToken(start=9, stop=12, text='Бог', pos='PROPN', feats=<Inan,Acc,Masc,Sing>, lemma='бог'),
 DocToken(start=13, stop=17, text='свет', pos='NOUN', feats=<Inan,Acc,Masc,Sing>, lemma='свет'),
 DocToken(start=18, stop=22, text='днем', pos='NOUN', feats=<Inan,Ins,Masc,Sing>, lemma='день'),
 DocToken(start=22, stop=23, text=',', pos='PUNCT', lemma=','),
 DocToken(start=24, stop=25, text='а', pos='CCONJ', lemma='а'),
 DocToken(start=26, stop=30, text='тьму', pos='NOUN', feats=<Inan,Acc,Fem,Sing>, lemma='тьма'),
 DocToken(start=31, stop=36, text='ночью', pos='NOUN', feats=<Inan,Ins,Fem,Sing>, lemma='ночь'),
 DocToken(start=36, stop=37, text='.', pos='PUNCT', lemma='.'),
 DocToken(start=38, stop=39, text='И', pos='CCONJ', lemma='и'),
 DocToken(start=40, stop=43, text='был', pos='AUX', feats=<Imp,Masc,Ind,Sing,Past,Fin,Act>, lemm

In [29]:
doc.parse_syntax(syntax_parser)

In [30]:
doc.tokens

[DocToken(stop=1, text='И', id='1_1', head_id='1_2', rel='cc', pos='CCONJ', lemma='и'),
 DocToken(start=2, stop=8, text='назвал', id='1_2', head_id='1_0', rel='root', pos='VERB', feats=<Perf,Masc,Ind,Sing,Past,Fin,Act>, lemma='назвать'),
 DocToken(start=9, stop=12, text='Бог', id='1_3', head_id='1_2', rel='obj', pos='PROPN', feats=<Inan,Acc,Masc,Sing>, lemma='бог'),
 DocToken(start=13, stop=17, text='свет', id='1_4', head_id='1_2', rel='obj', pos='NOUN', feats=<Inan,Acc,Masc,Sing>, lemma='свет'),
 DocToken(start=18, stop=22, text='днем', id='1_5', head_id='1_2', rel='xcomp', pos='NOUN', feats=<Inan,Ins,Masc,Sing>, lemma='день'),
 DocToken(start=22, stop=23, text=',', id='1_6', head_id='1_8', rel='punct', pos='PUNCT', lemma=','),
 DocToken(start=24, stop=25, text='а', id='1_7', head_id='1_8', rel='cc', pos='CCONJ', lemma='а'),
 DocToken(start=26, stop=30, text='тьму', id='1_8', head_id='1_9', rel='nsubj', pos='NOUN', feats=<Inan,Acc,Fem,Sing>, lemma='тьма'),
 DocToken(start=31, stop=36,

In [31]:
doc.sents[0].syntax.print()

        ┌► И      cc
┌─┌─┌─┌─└─ назвал 
│ │ │ └──► Бог    obj
│ │ └────► свет   obj
│ └──────► днем   xcomp
│     ┌──► ,      punct
│     │ ┌► а      cc
│   ┌─└─└─ тьму   nsubj
│   └►└─── ночью  orphan
└────────► .      punct


In [45]:
doc.tag_ner(ner_tagger)

In [46]:
doc.spans

[]

In [47]:
doc.ner

NERMarkup(
    text='И назвал Бог свет днем, а тьму ночью. И был вечер, и было утро: день один.',
    spans=[]
)

In [40]:
doc.ner.print()

И назвал Бог свет днем, а тьму ночью. И был вечер, и было утро: день 
один.


In [44]:
dir(doc)

['__annotations__',
 '__attributes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_pretty_',
 'as_json',
 'clear_envelopes',
 'envelop_sent_spans',
 'envelop_sent_tokens',
 'envelop_span_tokens',
 'from_json',
 'morph',
 'ner',
 'parse_syntax',
 'segment',
 'sents',
 'spans',
 'syntax',
 'tag_morph',
 'tag_ner',
 'text',
 'tokens']